#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример архитектуры Encoder-Decoder с использованием RNN. Обсудите концепцию teacher forcing.

In [ ]:
import torch as th
import torch.nn as nn

In [ ]:
n_ru_tokens = 100
batch_size = 16
ru_seq_len = 10

n_en_tokens = 200
en_seq_len = 20

ru = th.randint(0, n_ru_tokens, size=(batch_size, ru_seq_len))
en = th.randint(0, n_en_tokens, size=(batch_size, en_seq_len))

In [ ]:
encoder_embedding_dim = 32
encoder_hidden_dim = 64
encoder = nn.Sequential(
    nn.Embedding(n_ru_tokens, encoder_embedding_dim, padding_idx=0),
    nn.GRU(encoder_embedding_dim, encoder_hidden_dim, batch_first=True)
)
encoding = encoder(ru)[-1].squeeze()
encoding.shape

torch.Size([16, 64])

In [ ]:
EN_SOS_IDX = 2

class Decoder(nn.Module):
  def __init__(self, n_tokens, embedding_dim, hidden_size, seq_len):
    super().__init__()
    self.embedding = nn.Embedding(n_tokens, embedding_dim, padding_idx=0)
    self.gru = nn.GRUCell(embedding_dim, hidden_size)
    self.fc = nn.Linear(hidden_size, n_tokens)
    self.seq_len = seq_len

  def forward(self, encoding, labels: th.Tensor | None = None):
    batch_size = encoding.size(0)
    decoder_input = th.full(size=(batch_size, ), fill_value=EN_SOS_IDX ) # b

    decoder_h = encoding
    outputs = []
    for i in range(self.seq_len):
      emb = self.embedding(decoder_input) # b x e
      decoder_h = self.gru(emb, decoder_h) # b x h
      token_predictions = self.fc(decoder_h) # b x t

      if labels:
        decoder_input = labels[:, i]
      else:
        decoder_input = token_predictions.argmax(dim=-1).detach() # b

      outputs.append(token_predictions)
    outputs = th.stack(outputs, dim=1)
    return outputs

In [ ]:
decoder_embedding_dim = 32
decoder_hidden_dim = 64

decoder = Decoder(n_tokens=n_en_tokens, embedding_dim=decoder_embedding_dim,
                  hidden_size=decoder_hidden_dim, seq_len=en_seq_len)
decoder(encoding).shape

torch.Size([16, 20, 200])

In [ ]:
en.shape

torch.Size([16, 20])

## Задачи для самостоятельного решения

### 1.

<p class="task" id="1"></p>

1\. Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.

- [x] Проверено на семинаре

In [ ]:
import pandas as pd

train_data = pd.read_json('/content/RuBQ_2.0_train.json')
train_data.head()

,uid,question_text,query,answer_text,question_uris,question_props,answers,paragraphs_uids,tags,RuBQ_version,question_eng
0,0,Что может вызвать цунами?,SELECT ?answer \nWHERE {\n wd:Q8070 wdt:P828 ...,Землетрясение,[http://www.wikidata.org/entity/Q8070],[wdt:P828],"[{'type': 'uri', 'label': 'землетрясение', 'va...","{'with_answer': [35622], 'all_related': [35622...",[1-hop],1,What can cause a tsunami?
1,1,Кто написал роман «Хижина дяди Тома»?,SELECT ?answer \nWHERE {\n wd:Q2222 wdt:P50 ?...,Г. Бичер-Стоу,[http://www.wikidata.org/entity/Q2222],[wdt:P50],"[{'type': 'uri', 'label': 'Гарриет Бичер-Стоу'...","{'with_answer': [35652], 'all_related': [35652...",[1-hop],1,"Who wrote the novel ""uncle Tom's Cabin""?"
2,2,Кто автор пьесы «Ромео и Джульетта»?,SELECT ?answer \nWHERE {\n wd:Q83186 wdt:P50 ...,Шекспир,[http://www.wikidata.org/entity/Q83186],[wdt:P50],"[{'type': 'uri', 'label': 'Уильям Шекспир', 'v...","{'with_answer': [35676, 35677], 'all_related':...",[1-hop],1,"Who is the author of the play ""Romeo and Juliet""?"
3,3,Как называется столица Румынии?,SELECT ?answer \nWHERE {\n wd:Q218 wdt:P36 ?a...,Бухарест,[http://www.wikidata.org/entity/Q218],[wdt:P36],"[{'type': 'uri', 'label': 'Бухарест', 'value':...","{'with_answer': [35702, 35703], 'all_related':...",[1-hop],1,What is the name of the capital of Romania?
4,5,На каком инструменте играл Джимми Хендрикс?,SELECT ?answer \nWHERE {\n wd:Q5928 wdt:P1303...,Гитара,[http://www.wikidata.org/entity/Q5928],[wdt:P1303],"[{'type': 'uri', 'label': 'гитара', 'value': '...","{'with_answer': [35728, 35727], 'all_related':...",[1-hop],1,What instrument did Jimi Hendrix play?


In [ ]:
import re

def preprocess(text):
  text = text.lower()
  text = re.sub(r'[^a-zа-яё\s]','',text)
  words = text.split()
  return ' '.join(words)

In [ ]:
train_data['question_text'] = train_data['question_text'].apply(preprocess)
train_data['question_text'].head()

,question_text
0,что может вызвать цунами
1,кто написал роман хижина дяди тома
2,кто автор пьесы ромео и джульетта
3,как называется столица румынии
4,на каком инструменте играл джимми хендрикс


In [ ]:
sents_ru_train = train_data['question_text'].to_list()
sents_ru_train[:5]

['что может вызвать цунами',
 'кто написал роман хижина дяди тома',
 'кто автор пьесы ромео и джульетта',
 'как называется столица румынии',
 'на каком инструменте играл джимми хендрикс']

In [ ]:
train_data['question_eng'] = train_data['question_eng'].apply(preprocess)
train_data['question_eng'].head()

,question_eng
0,what can cause a tsunami
1,who wrote the novel uncle toms cabin
2,who is the author of the play romeo and juliet
3,what is the name of the capital of romania
4,what instrument did jimi hendrix play


In [ ]:
sents_eng_train = train_data['question_eng'].to_list()
sents_eng_train[:5]

['what can cause a tsunami',
 'who wrote the novel uncle toms cabin',
 'who is the author of the play romeo and juliet',
 'what is the name of the capital of romania',
 'what instrument did jimi hendrix play']

In [ ]:
test_data = pd.read_json('/content/RuBQ_2.0_test.json')
test_data.head()

,uid,question_text,query,answer_text,question_uris,question_props,answers,paragraphs_uids,tags,RuBQ_version,question_eng
0,4,Какой стране принадлежит знаменитый остров Пасхи?,SELECT ?answer \nWHERE {\n wd:Q14452 wdt:P17 ...,Чили,[http://www.wikidata.org/entity/Q14452],[wdt:P17],"[{'type': 'uri', 'label': 'Чили', 'value': 'ht...","{'with_answer': [10785, 10782, 10783], 'all_re...",[1-hop],1,Which country does the famous Easter island be...
1,7,С какой музыкальной группой неразрывно связано...,SELECT ?answer \nWHERE {\n wd:Q128121 wdt:P36...,'Роллинг Стоунз',[http://www.wikidata.org/entity/Q128121],[wdt:P361],"[{'type': 'uri', 'label': 'The Rolling Stones'...","{'with_answer': [53040, 53037, 53038, 53039], ...",[1-hop],1,Which music group is Mick Jagger's name inextr...
2,14,Где находится Летний сад?,SELECT ?answer \nWHERE {\n wd:Q1229234 wdt:P1...,Санкт-Петербург,[http://www.wikidata.org/entity/Q1229234],[wdt:P131],"[{'type': 'uri', 'label': 'Санкт-Петербург', '...","{'with_answer': [25080, 25067], 'all_related':...",[1-hop],1,Where is the Summer garden?
3,22,Какой город является столицей Туркмении?,SELECT ?answer \nWHERE {\n wd:Q874 wdt:P36 ?a...,Ашхабад,[http://www.wikidata.org/entity/Q874],[wdt:P36],"[{'type': 'uri', 'label': 'Ашхабад', 'value': ...","{'with_answer': [14910], 'all_related': [11522...",[1-hop],1,Which city is the capital of Turkmenistan?
4,25,В каком городе издавалась с 1857 г. А. Герцено...,SELECT ?answer \nWHERE {\n wd:Q2533402 wdt:P1...,Лондон,[http://www.wikidata.org/entity/Q2533402],[wdt:P159],"[{'type': 'uri', 'label': 'Женева', 'value': '...","{'with_answer': [], 'all_related': [53072, 530...",[1-hop],1,In which city was the first Russian revolution...


In [ ]:
test_data['question_text'] = test_data['question_text'].apply(preprocess)
test_data['question_text'].head()

,question_text
0,какой стране принадлежит знаменитый остров пасхи
1,с какой музыкальной группой неразрывно связано...
2,где находится летний сад
3,какой город является столицей туркмении
4,в каком городе издавалась с г а герценом и н о...


In [ ]:
sents_ru_test = test_data['question_text'].to_list()
sents_ru_test[:5]

['какой стране принадлежит знаменитый остров пасхи',
 'с какой музыкальной группой неразрывно связано имя мика джаггера',
 'где находится летний сад',
 'какой город является столицей туркмении',
 'в каком городе издавалась с г а герценом и н огаревым первая российская революционная газета колокол']

In [ ]:
test_data['question_eng'] = test_data['question_eng'].apply(preprocess)
test_data['question_eng'].head()

,question_eng
0,which country does the famous easter island be...
1,which music group is mick jaggers name inextri...
2,where is the summer garden
3,which city is the capital of turkmenistan
4,in which city was the first russian revolution...


In [ ]:
sents_eng_test = test_data['question_eng'].to_list()
sents_eng_test[:5]

['which country does the famous easter island belong to',
 'which music group is mick jaggers name inextricably linked to',
 'where is the summer garden',
 'which city is the capital of turkmenistan',
 'in which city was the first russian revolutionary newspaper kolokol published since by a herzen and n ogarev']

In [ ]:
len(train_data), len(test_data)

(2330, 580)

In [ ]:
spec = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']

In [ ]:
class Vocab:
  def __init__(self, sents, special_tok=None):
    self.word2idx = {}
    self.idx2word = {}
    self.n_tokens = 0
    self.special_tok = special_tok
    self._build_vocab(sents)

  def _build_vocab(self, sents):
    for token in self.special_tok:
      if token not in self.word2idx:
        self.word2idx[token] = self.n_tokens
        self.idx2word[self.n_tokens] = token
        self.n_tokens += 1

    for sent in sents:
      for word in sent.split():
        if word not in self.word2idx:
            self.word2idx[word] = self.n_tokens
            self.idx2word[self.n_tokens] = word
            self.n_tokens += 1

  def __len__(self):
    return self.n_tokens

  def __getitem__(self, token):
    return self.word2idx.get(token, self.word2idx['<UNK>'])

In [ ]:
ru_vocab = Vocab(sents_ru_train, special_tok=spec)
en_vocab = Vocab(sents_eng_train, special_tok=spec)

ru_vocab.word2idx

{'<PAD>': 0,
 '<UNK>': 1,
 '<SOS>': 2,
 '<EOS>': 3,
 'что': 4,
 'может': 5,
 'вызвать': 6,
 'цунами': 7,
 'кто': 8,
 'написал': 9,
 'роман': 10,
 'хижина': 11,
 'дяди': 12,
 'тома': 13,
 'автор': 14,
 'пьесы': 15,
 'ромео': 16,
 'и': 17,
 'джульетта': 18,
 'как': 19,
 'называется': 20,
 'столица': 21,
 'румынии': 22,
 'на': 23,
 'каком': 24,
 'инструменте': 25,
 'играл': 26,
 'джимми': 27,
 'хендрикс': 28,
 'какой': 29,
 'стране': 30,
 'принадлежит': 31,
 'остров': 32,
 'таити': 33,
 'создал': 34,
 'зимний': 35,
 'дворец': 36,
 'в': 37,
 'санктпетербурге': 38,
 'режиссер': 39,
 'фильма': 40,
 'бриллиантовая': 41,
 'рука': 42,
 'принадлежат': 43,
 'канарские': 44,
 'острова': 45,
 'географический': 46,
 'объект': 47,
 'лимпопо': 48,
 'какое': 49,
 'современное': 50,
 'государство': 51,
 'занимает': 52,
 'территорию': 53,
 'которой': 54,
 'находился': 55,
 'древний': 56,
 'ассирийский': 57,
 'город': 58,
 'ниневия': 59,
 'по': 60,
 'шкале': 61,
 'измеряют': 62,
 'силу': 63,
 'землетрясен

In [ ]:
en_vocab.word2idx

{'<PAD>': 0,
 '<UNK>': 1,
 '<SOS>': 2,
 '<EOS>': 3,
 'what': 4,
 'can': 5,
 'cause': 6,
 'a': 7,
 'tsunami': 8,
 'who': 9,
 'wrote': 10,
 'the': 11,
 'novel': 12,
 'uncle': 13,
 'toms': 14,
 'cabin': 15,
 'is': 16,
 'author': 17,
 'of': 18,
 'play': 19,
 'romeo': 20,
 'and': 21,
 'juliet': 22,
 'name': 23,
 'capital': 24,
 'romania': 25,
 'instrument': 26,
 'did': 27,
 'jimi': 28,
 'hendrix': 29,
 'country': 30,
 'owns': 31,
 'island': 32,
 'tahiti': 33,
 'created': 34,
 'winter': 35,
 'palace': 36,
 'in': 37,
 'st': 38,
 'petersburg': 39,
 'director': 40,
 'film': 41,
 'diamond': 42,
 'hand': 43,
 'which': 44,
 'does': 45,
 'canary': 46,
 'islands': 47,
 'belong': 48,
 'to': 49,
 'geographical': 50,
 'feature': 51,
 'called': 52,
 'limpopo': 53,
 'modern': 54,
 'state': 55,
 'occupies': 56,
 'territory': 57,
 'where': 58,
 'ancient': 59,
 'assyrian': 60,
 'city': 61,
 'nineveh': 62,
 'was': 63,
 'located': 64,
 'scale': 65,
 'used': 66,
 'measure': 67,
 'strength': 68,
 'earthquakes':

In [ ]:
ru_vocab.word2idx['<UNK>']

1

In [ ]:
ru_vocab.word2idx['<EOS>']

3

In [ ]:
ru_vocab.word2idx['<PAD>']
en_vocab.word2idx['<PAD>']

0

### 2.

<p class="task" id="2"></p>

2\. Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [x] Проверено на семинаре

In [ ]:
import torch as th

In [ ]:
class RuEnDataset:
  def __init__(self, X, y, vocab_x, vocab_y, max_len=None):
    self.pairs = list(zip(X, y))
    self.X = X
    self.y = y
    self.vocab_x = vocab_x
    self.vocab_y = vocab_y
    self.max_len = max_len
    self.pad_idx_x = vocab_x['<PAD>']
    self.pad_idx_y = vocab_y['<PAD>']

  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, idx):
    rus, engs = self.pairs[idx]
    rus = rus.split()
    engs = engs.split()

    x_idxs = [self.vocab_x['<SOS>']] + [self.vocab_x[w] for w in rus] + [self.vocab_x['<EOS>']]
    y_idxs = [self.vocab_y['<SOS>']] + [self.vocab_y[w] for w in engs] + [self.vocab_y['<EOS>']]

    if len(x_idxs) < self.max_len:
      x_idxs = x_idxs + [self.vocab_x['<PAD>']] * (self.max_len - len(x_idxs))
    else:
      x_idxs = x_idxs[:self.max_len]
      x_idxs[-1] = self.vocab_x['<EOS>']


    if len(y_idxs) < self.max_len:
      y_idxs = y_idxs + [self.vocab_y['<PAD>']] * (self.max_len - len(y_idxs))
    else:
      y_idxs = y_idxs[:self.max_len]
      y_idxs[-1] = self.vocab_y['<EOS>']

    return th.tensor(x_idxs), th.tensor(y_idxs)

In [ ]:
train_dataset =  RuEnDataset(sents_ru_train, sents_eng_train, ru_vocab, en_vocab, max_len=20)
test_dataset =  RuEnDataset(sents_ru_test, sents_eng_test, ru_vocab, en_vocab, max_len=20)

In [ ]:
x, y = zip(*[train_dataset[i] for i in range(5)])
x = th.stack(x)
y = th.stack(y)

print(x.shape, y.shape)

torch.Size([5, 20]) torch.Size([5, 20])


In [ ]:
x[0]

tensor([2, 4, 5, 6, 7, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### 3.

<p class="task" id="4"></p>

3\. Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://adeveloperdiary.com/assets/img/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.webp)

- [ ] Проверено на семинаре

In [ ]:
import torch as th
import torch.nn as nn

class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx):
    super().__init__()
    self.embed = nn.Sequential(
        nn.Embedding(vocab_size, embedding_dim, padding_idx),
        nn.Dropout(0.3)
    )
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

  def forward(self, x): # x - (b, seq_len)
    embed = self.embed(x) # (b, seq_len, emb_dim)
    _, (h, cell) = self.rnn(embed) # h - (num_layers, b, hid_dim)
    return h

In [ ]:
embedding_dim = 256
hidden_dim = 512
padding_idx = ru_vocab['<PAD>']

encoder = Encoder(len(ru_vocab), embedding_dim, hidden_dim, padding_idx)

#batch = x[:16] # (b, seq_len) - (16, 20)
batch_idxs = range(16)
batch_x = th.stack([train_dataset[i][0] for i in batch_idxs])  # х русс
#batch_y = th.stack([train_dataset[i][1] for i in batch_idxs]) англ

hidden_state = encoder(batch_x)
print(hidden_state.shape)

torch.Size([1, 16, 512])


### 4.

<p class="task" id="5"></p>

4\. Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://adeveloperdiary.com/assets/img/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.webp)

- [ ] Проверено на семинаре

In [ ]:
len(en_vocab)

4234

In [ ]:
import random

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, padding_idx):
    super().__init__()
    self.embed = nn.Sequential(
        nn.Embedding(vocab_size, embedding_dim, padding_idx),
        nn.Dropout(0.3)
    )
    #nn.Embedding(vocab_size, embedding_dim, padding_idx)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, vocab_size)
    self.sos_idx = en_vocab['<SOS>']

  def forward(self, hidden, max_len=20, labels = None, teacher_forcing_ratio=0.5):
    batch_size = hidden.size(1) # т.к. torch.Size([1, 16, 512]) - [1, batch_size, hidden_dim]
    # переделываю enc hid в формат (h_0, c_0) для lstm
    h_0 = hidden
    c_0 = th.zeros_like(h_0) # cell state

    decoder_input = th.full(size=(batch_size, 1), fill_value=self.sos_idx, device=hidden.device)

    teacher_forcing_ratio = 0.5 if self.training else 0.0
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    outputs = []
    for i in range(max_len):
      embed = self.embed(decoder_input) # b,1,e
      output, (h_n, c_n) = self.rnn(embed, (h_0, c_0)) # пропускаю ч/з лстм с текущим состоянием

      h_0, c_0 = h_n, c_n # обновляю сост для след шага

      token_predictions = self.fc(output.squeeze(1)) # b, vocab_size
      outputs.append(token_predictions)

      if use_teacher_forcing and labels is not None:
        decoder_input = labels[:, i].unsqueeze(1).to(hidden.device)
      else:
        decoder_input = token_predictions.argmax(dim=-1).unsqueeze(1)

    outputs = th.stack(outputs, dim=1) # [batch_size, seq_len, vocab_size]
    return outputs

In [ ]:
decoder = Decoder(len(en_vocab), embedding_dim, hidden_dim, padding_idx=en_vocab['<PAD>'])

output = decoder(hidden_state)
print(output.shape)

torch.Size([16, 20, 4234])


### 5.

<p class="task" id="5"></p>

5\. Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Настройте модель, решив задачу классификации. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, x, trg=None, max_len=20):
    hidden = self.encoder(x)

    if trg is not None:
      max_len = trg.size(1)

    output = self.decoder(hidden, labels=trg, max_len=max_len)
    return output

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x, trg=None, max_len=20, teacher_forcing_ratio=0.5):
        hidden = self.encoder(x)

        if trg is not None:
          max_len = trg.size(1)

        output = self.decoder(hidden, labels=trg, max_len=max_len, teacher_forcing_ratio=teacher_forcing_ratio)
        return output


In [ ]:
def loss_func(outputs, targets, pad_idx):
  outputs = outputs.view(-1, outputs.size(-1))
  targets = targets.view(-1)
  mask = targets != pad_idx
  outputs = outputs[mask]
  targets = targets[mask]

  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, reduction='mean')
  return criterion(outputs, targets)

def calc_accuracy(outputs, targets, pad_idx):
  with th.no_grad():
    preds = outputs.argmax(dim=-1)
    mask = targets!=pad_idx
    correct = (preds[mask]==targets[mask]).float().sum()
    total = mask.float().sum()

    return (correct/total).item()

def show_example(model, src, device, max_len=20):
  model.eval()
  with th.no_grad():
    src = src.to(device)
    output = model(src, max_len=max_len)
    pred_indxs = output.argmax(dim=-1).squeeze(0).cpu().numpy()
    src_words = [ru_vocab.idx2word[idx] for idx in src.squeeze(0).cpu().numpy()
                if idx not in {ru_vocab['<PAD>'], ru_vocab['<SOS>'], ru_vocab['<EOS>']}] # фильтрую служебные токены для вывода
    pred_words = [en_vocab.idx2word[idx] for idx in pred_indxs
                  if idx not in {en_vocab['<PAD>'], en_vocab['<SOS>'], en_vocab['<EOS>']}]

    print(f'Пример: {" ".join(src_words)}')
    print(f'Предсказанное: {" ".join(pred_words)}')

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
def train_model(model, train_loader, val_loader, n_epochs, lr, device):
  optimizer = th.optim.Adam(model.parameters(), lr=lr)
  pad_idx = en_vocab['<PAD>']
  model = model.to(device)

  for epoch in range(n_epochs):
    model.train()
    total_loss, total_acc, n_batches = 0, 0, 0

    for src, trg in train_loader:
      src, trg = src.to(device), trg.to(device)

      optimizer.zero_grad()
      outputs = model(src, trg=trg)

      loss = loss_func(outputs, trg, pad_idx)
      accuracy = calc_accuracy(outputs, trg, pad_idx)

      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      total_acc += accuracy
      n_batches += 1

    avg_loss = total_loss / n_batches
    avg_acc = total_acc / n_batches

    model.eval()
    test_loss, test_acc, n_val = 0, 0, 0

    with th.no_grad():
      for src, trg in val_loader:
        src, trg = src.to(device), trg.to(device)
        outputs = model(src, trg=trg)

        loss = loss_func(outputs, trg, pad_idx)
        accuracy = calc_accuracy(outputs, trg, pad_idx)

        test_loss += loss.item()
        test_acc += accuracy
        n_val += 1

    test_avg_loss = test_loss / n_val
    test_avg_acc = test_acc / n_val


    print(f'\nEpoch: {epoch+1}')
    show_example(model, train_dataset[0][0].unsqueeze(0), device)
    print(f'train loss: {avg_loss:.3f} | train_acc: {avg_acc*100:.2f}%')
    print(f'test loss: {test_avg_loss:.3f} | test_acc: {test_avg_acc*100:.2f}%')

def calc_bleu(model, test_dataset, device, max_len=20):
  model.eval()
  references = []
  hypotheses = []

  with th.no_grad():
    for i in range(len(test_dataset)):
      src, trg = test_dataset[i]
      src = src.unsqueeze(0).to(device)

      output = model(src, max_len=max_len)
      pred_indxs = output.argmax(dim=-1).squeeze(0).cpu().numpy()

      trg_words = [en_vocab.idx2word[idx] for idx in trg.numpy()]  # полные последовательности
      pred_words = [en_vocab.idx2word[idx] for idx in pred_indxs]

      trg_words = [w for w in trg_words if w != '<PAD>']  # удаляю пад токены
      pred_words = [w for w in pred_words if w != '<PAD>']

      if '<EOS>' in pred_words:
        pred_words = pred_words[:pred_words.index('<EOS>')+1]

      references.append([trg_words])
      hypotheses.append(pred_words)

  bleu_score = corpus_bleu(references, hypotheses)
  print(f'BLEU score: {bleu_score:.4f}')

In [ ]:
embedding_dim = 128
hidden_dim = 256
padding_idx = en_vocab['<PAD>']

encoder = Encoder(
    vocab_size=len(ru_vocab),
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    padding_idx=ru_vocab['<PAD>']
)

decoder = Decoder(
    vocab_size=len(en_vocab),
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    padding_idx=padding_idx
)

model = EncoderDecoder(encoder, decoder)

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
batch_size = 64
n_epochs = 10
lr = 0.001

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = EncoderDecoder(encoder, decoder).to(device)
train_model(model, train_loader, test_loader, n_epochs=n_epochs, lr=lr, device=device)


Epoch: 1
Пример: что может вызвать цунами
Предсказанное: what what the the
train loss: 6.257 | train_acc: 17.85%
test loss: 5.040 | test_acc: 27.57%

Epoch: 2
Пример: что может вызвать цунами
Предсказанное: what is the the the of the
train loss: 4.839 | train_acc: 29.01%
test loss: 4.866 | test_acc: 28.14%

Epoch: 3
Пример: что может вызвать цунами
Предсказанное: what is the the the of the
train loss: 4.639 | train_acc: 30.61%
test loss: 4.842 | test_acc: 28.14%

Epoch: 4
Пример: что может вызвать цунами
Предсказанное: what is the the of the of the
train loss: 4.532 | train_acc: 32.03%
test loss: 4.850 | test_acc: 27.98%

Epoch: 5
Пример: что может вызвать цунами
Предсказанное: what is the the of the
train loss: 4.434 | train_acc: 33.25%
test loss: 4.871 | test_acc: 29.77%

Epoch: 6
Пример: что может вызвать цунами
Предсказанное: what is the the the of
train loss: 4.421 | train_acc: 32.27%
test loss: 4.882 | test_acc: 29.18%

Epoch: 7
Пример: что может вызвать цунами
Предсказанное: wh

In [ ]:
calc_bleu(model, test_dataset, device)

BLEU score: 0.0693


### 6.

<p class="task" id="6"></p>

6*\. Создайте и обучите модель машинного перевода, используя архитектуру Encoder-Decoder на основе RNN с использованием механизма аддитивного внимания. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

Сгенерируйте перевод при помощи обученной модели и визуализируйте матрицу внимания, в которой отображено, на какие слова из исходного предложения модель обращала внимание при генерации очередного слова в переводе.

- [ ] Проверено на семинаре